In [1]:
import pandas as pd

In [3]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

## El método `concat` Parte 1

In [4]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [11]:
len(week1)
week1

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
5,213,8
6,600,1
7,503,5
8,71,3
9,174,3


In [13]:
len(week2)
week2

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3
5,310,5
6,761,2
7,443,5
8,729,9
9,741,8


In [8]:
len(pd.concat([week1, week2]))

500

In [10]:
two_weeks = pd.concat([week1, week2])
two_weeks.tail()

,Customer ID,Food ID
245,783,10
246,556,10
247,547,9
248,252,9
249,249,6


## El método `concat()` Parte 2

In [14]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [18]:
sales = pd.concat([week1, week2], keys = ["A", "B"])
sales


Customer ID  Food ID
A 0            537        9
  1             97        4
  2            658        1
  3            202        2
  4            155        9
  5            213        8
  6            600        1
  7            503        5
  8             71        3
  9            174        3
  10           961        9
  11           966        5
  12           641        4
  13           288        2
  14           149        4
  15           954        2
  16           147        9
  17           155        1
  18           550        6
  19           101        7
  20           549        6
  21            75        6
  22            78        7
  23           514        5
  24           833        7
  25           329        8
  26           586       10
  27           341        1
  28           519       10
  29           680        9
...            ...      ...
B 220          495        2
  221          794        6
  222          734       10
  223          540        3
  224          735        2
  225          940        8
  226          571        7
  227          888        2
  228          664        6
  229          343        7
  230          143        4
  231          505        3
  232           54        8
  233          959        2
  234          367        8
  235          883        8
  236          251        9
  237          855        4
  238          233        3
  239          559       10
  240          734        1
  241          677        3
  242          276        4
  243           45        8
  244          945        4
  245          783       10
  246          556       10
  247          547        9
  248          252        9
  249          249        6

[500 rows x 2 columns]

In [20]:
sales.loc[("B", 240), "Customer ID"]

734

## El método `append`

In [21]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [31]:
week1.tail()

,Customer ID,Food ID
245,413,9
246,926,6
247,134,3
248,396,6
249,535,10


In [29]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [32]:
sales = week2.append(week1)
sales

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3
5,310,5
6,761,2
7,443,5
8,729,9
9,741,8


In [33]:
sales = week2.append(week1, ignore_index=True)
sales

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3
5,310,5
6,761,2
7,443,5
8,729,9
9,741,8


## Inner Joins, Parte 1

In [34]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [35]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [37]:
week2.head(2)

,Customer ID,Food ID
0,688,10
1,813,7


Vamos a obtener todos los platos consumidos por cada cliente durante las dos semanas

In [28]:
week1.merge(week2, how = "inner", on = "Customer ID", suffixes = [" - A", " - B"])

,Customer ID,Food ID - A,Food ID - B
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
5,550,6,7
6,101,7,4
7,75,6,4
8,77,1,7
9,77,9,7


In [38]:
week1[week1["Customer ID"] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [39]:
week2[week2["Customer ID"] == 155]

,Customer ID,Food ID
208,155,3


## Inner Joins, Parte 2

In [40]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [66]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [67]:
week2.head(2)

,Customer ID,Food ID
0,688,10
1,813,7


In [68]:
week1.merge(week2, how = "inner", on = ["Customer ID", "Food ID"])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


In [71]:
week1[week1["Customer ID"] == 578]

,Customer ID,Food ID
224,578,5


In [72]:
week2[week2["Customer ID"] == 578]

,Customer ID,Food ID
29,578,5
189,578,5


## Outer Joins

In [77]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [78]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [79]:
week2.head(2)

,Customer ID,Food ID
0,688,10
1,813,7


In [41]:
merged = week1.merge(week2, how = "outer", on = "Customer ID", suffixes = [" - Week 1", " - Week 2"],
            indicator = True)

In [42]:
merged["_merge"].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [43]:
merged


,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
5,155,1.0,3.0,both
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
8,503,5.0,8.0,both
9,503,5.0,9.0,both


In [44]:
mask = merged["_merge"].isin(["left_only", "right_only"])
merged[mask]

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
10,71,3.0,NaN,left_only
11,71,8.0,NaN,left_only
12,174,3.0,NaN,left_only
13,961,9.0,NaN,left_only
14,966,5.0,NaN,left_only


In [46]:
mask = merged["_merge"].isin(["both"])
merged[mask]

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,537,9.0,5.0,both
4,155,9.0,3.0,both
5,155,1.0,3.0,both
8,503,5.0,8.0,both
9,503,5.0,9.0,both
21,550,6.0,7.0,both
22,101,7.0,4.0,both
24,75,6.0,4.0,both
41,77,1.0,7.0,both
42,77,9.0,7.0,both


## Left Joins

In [47]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [48]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [49]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [52]:
week1 = week1.merge(foods, how = "left", on = "Food ID", sort = True)

Vamos a obtener los nombres de los alimentos y su precio consumidos en la semana 1 de forma ordenada

In [51]:
week1.head()

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99


## Los parámetros `left_on` y `right_on`

In [53]:
week1 = pd.read_csv("pandas/Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("pandas/Restaurant - Customers.csv")
foods = pd.read_csv("pandas/Restaurant - Foods.csv")

In [54]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [55]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [57]:
week2 = week2.merge(customers, how = "left", left_on = "Customer ID", right_on = "ID", sort = True)

In [59]:
week2.head()

,Customer ID,Food ID,ID_x,First Name_x,Last Name_x,Gender_x,Company_x,Occupation_x,ID_y,First Name_y,Last Name_y,Gender_y,Company_y,Occupation_y
0,8,6,8,Frances,Adams,Female,Dabshots,Developer III,8,Frances,Adams,Female,Dabshots,Developer III
1,13,2,13,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer,13,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,21,Albert,Burns,Male,Rhynoodle,Junior Executive,21,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,24,Donna,Thomas,Female,Jaxbean,Chief Design Engineer,24,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,27,Jessica,Bennett,Female,Twitternation,Account Executive,27,Jessica,Bennett,Female,Twitternation,Account Executive


In [61]:
week2 = pd.read_csv("pandas/Restaurant - Week 2 Sales.csv")
week2 = week2.merge(customers, how = "left", left_on = "Customer ID", right_on = "ID", sort = True).drop("ID", axis = "columns")
week2.head()

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,Frances,Adams,Female,Dabshots,Developer III
1,13,2,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,Jessica,Bennett,Female,Twitternation,Account Executive


## Merging en base a Indexes con los parámetros `left_index` y `right_index`

In [118]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv", index_col = "ID")
foods = pd.read_csv("Restaurant - Foods.csv", index_col = "Food ID")

In [119]:
foods.head(3)

,Food Item,Price
Food ID,,
1,Sushi,3.99
2,Burrito,9.99
3,Taco,2.99


In [125]:
sales = week1.merge(customers, how = "left", left_on = "Customer ID", right_index = True)
sales = sales.merge(foods, how = "left", left_on = "Food ID", right_index = True)
sales.head(3)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99


In [126]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [127]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [129]:
week1.merge(week2, how = "left", left_index = True, right_index = True, suffixes = [" - Week 1", " - Week 2"])

,Customer ID - Week 1,Food ID - Week 1,Customer ID - Week 2,Food ID - Week 2
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
5,213,8,310,5
6,600,1,761,2
7,503,5,443,5
8,71,3,729,9
9,174,3,741,8


## El método `join`

In [3]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
foods = pd.read_csv("Restaurant - Foods.csv")
satisfaction = pd.read_csv("Restaurant - Week 1 Satisfaction.csv")

In [4]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [5]:
satisfaction.head(3)

,Satisfaction Rating
0,2
1,7
2,3


In [9]:
week1.merge(satisfaction, how = "left", left_index = True, right_index = True).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


In [8]:
week1.join(satisfaction).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


## El método `merge`

In [10]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
foods = pd.read_csv("Restaurant - Foods.csv")
satisfaction = pd.read_csv("Restaurant - Week 1 Satisfaction.csv")

In [11]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [12]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [13]:
pd.merge(week1, customers, how = "left", left_on = "Customer ID", right_on = "ID")

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
5,213,8,213,Keith,Foster,Male,Gigashots,VP Quality Control
6,600,1,600,Sandra,Bailey,Female,Jaxnation,Sales Associate
7,503,5,503,Justin,Daniels,Male,Oloo,Environmental Tech
8,71,3,71,Carolyn,Reyes,Female,Twimbo,Safety Technician IV
9,174,3,174,Anna,Miller,Female,Innojam,Developer I
